In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
tables = [
r'SNL_18650_LFP_15C_0-100_0.5-1C_a',
r'SNL_18650_LFP_15C_0-100_0.5-1C_b',
r'SNL_18650_LFP_15C_0-100_0.5-2C_a',
r'SNL_18650_LFP_15C_0-100_0.5-2C_b',
r'SNL_18650_LFP_25C_0-100_0.5-0.5C_a',
r'SNL_18650_LFP_25C_0-100_0.5-1C_a',
r'SNL_18650_LFP_25C_0-100_0.5-1C_b',
r'SNL_18650_LFP_25C_0-100_0.5-1C_c',
r'SNL_18650_LFP_25C_0-100_0.5-1C_d',
r'SNL_18650_LFP_25C_0-100_0.5-2C_a',
r'SNL_18650_LFP_25C_0-100_0.5-2C_b',
r'SNL_18650_LFP_25C_0-100_0.5-3C_a',
r'SNL_18650_LFP_25C_0-100_0.5-3C_b',
r'SNL_18650_LFP_25C_0-100_0.5-3C_c',
r'SNL_18650_LFP_25C_0-100_0.5-3C_d',
r'SNL_18650_LFP_25C_20-80_0.5-0.5C_a',
r'SNL_18650_LFP_25C_20-80_0.5-0.5C_b',
r'SNL_18650_LFP_25C_20-80_0.5-0.5C_c',
r'SNL_18650_LFP_25C_20-80_0.5-0.5C_d',
r'SNL_18650_LFP_25C_20-80_0.5-3C_a',
r'SNL_18650_LFP_25C_40-60_0.5-0.5C_a',
r'SNL_18650_LFP_25C_40-60_0.5-0.5C_b',
r'SNL_18650_LFP_25C_40-60_0.5-3C_a',
r'SNL_18650_LFP_25C_40-60_0.5-3C_b',
r'SNL_18650_LFP_35C_0-100_0.5-1C_a',
r'SNL_18650_LFP_35C_0-100_0.5-1C_b',
r'SNL_18650_LFP_35C_0-100_0.5-1C_c',
r'SNL_18650_LFP_35C_0-100_0.5-1C_d',
r'SNL_18650_LFP_35C_0-100_0.5-2C_a',
r'SNL_18650_LFP_35C_0-100_0.5-2C_b',
r'SNL_18650_NCA_15C_0-100_0.5-1C_a',
r'SNL_18650_NCA_15C_0-100_0.5-1C_b',
r'SNL_18650_NCA_15C_0-100_0.5-2C_a',
r'SNL_18650_NCA_15C_0-100_0.5-2C_b',
r'SNL_18650_NCA_25C_0-100_0.5-0.5C_a',
r'SNL_18650_NCA_25C_0-100_0.5-0.5C_b',
r'SNL_18650_NCA_25C_0-100_0.5-1C_a',
r'SNL_18650_NCA_25C_0-100_0.5-1C_b',
r'SNL_18650_NCA_25C_0-100_0.5-1C_c',
r'SNL_18650_NCA_25C_0-100_0.5-1C_d',
r'SNL_18650_NCA_25C_0-100_0.5-2C_a',
r'SNL_18650_NCA_25C_0-100_0.5-2C_b',
r'SNL_18650_NCA_25C_20-80_0.5-0.5C_a',
r'SNL_18650_NCA_25C_20-80_0.5-0.5C_b',
r'SNL_18650_NCA_25C_20-80_0.5-0.5C_c',
r'SNL_18650_NCA_25C_20-80_0.5-0.5C_d',
r'SNL_18650_NCA_25C_40-60_0.5-0.5C_a',
r'SNL_18650_NCA_25C_40-60_0.5-0.5C_b',
r'SNL_18650_NCA_35C_0-100_0.5-1C_a',
r'SNL_18650_NCA_35C_0-100_0.5-1C_b',
r'SNL_18650_NCA_35C_0-100_0.5-1C_c',
r'SNL_18650_NCA_35C_0-100_0.5-1C_d',
r'SNL_18650_NCA_35C_0-100_0.5-2C_a',
r'SNL_18650_NCA_35C_0-100_0.5-2C_b',
r'SNL_18650_NMC_15C_0-100_0.5-1C_a',
r'SNL_18650_NMC_15C_0-100_0.5-1C_b',
r'SNL_18650_NMC_15C_0-100_0.5-2C_a',
r'SNL_18650_NMC_15C_0-100_0.5-2C_b',
r'SNL_18650_NMC_25C_0-100_0.5-0.5C_a',
r'SNL_18650_NMC_25C_0-100_0.5-0.5C_b',
r'SNL_18650_NMC_25C_0-100_0.5-1C_a',
r'SNL_18650_NMC_25C_0-100_0.5-1C_b',
r'SNL_18650_NMC_25C_0-100_0.5-1C_c',
r'SNL_18650_NMC_25C_0-100_0.5-1C_d',
r'SNL_18650_NMC_25C_0-100_0.5-2C_a',
r'SNL_18650_NMC_25C_0-100_0.5-2C_b',
r'SNL_18650_NMC_25C_0-100_0.5-3C_a',
r'SNL_18650_NMC_25C_0-100_0.5-3C_b',
r'SNL_18650_NMC_25C_0-100_0.5-3C_c',
r'SNL_18650_NMC_25C_0-100_0.5-3C_d',
r'SNL_18650_NMC_25C_20-80_0.5-0.5C_a',
r'SNL_18650_NMC_25C_20-80_0.5-0.5C_b',
r'SNL_18650_NMC_25C_20-80_0.5-0.5C_c',
r'SNL_18650_NMC_25C_20-80_0.5-0.5C_d',
r'SNL_18650_NMC_25C_20-80_0.5-3C_a',
r'SNL_18650_NMC_25C_20-80_0.5-3C_b',
r'SNL_18650_NMC_25C_40-60_0.5-0.5C_a',
r'SNL_18650_NMC_25C_40-60_0.5-0.5C_b',
r'SNL_18650_NMC_25C_40-60_0.5-3C_a',
r'SNL_18650_NMC_25C_40-60_0.5-3C_b',
r'SNL_18650_NMC_35C_0-100_0.5-1C_a',
r'SNL_18650_NMC_35C_0-100_0.5-1C_b',
r'SNL_18650_NMC_35C_0-100_0.5-1C_c',
r'SNL_18650_NMC_35C_0-100_0.5-1C_d',
r'SNL_18650_NMC_35C_0-100_0.5-2C_a',
r'SNL_18650_NMC_35C_0-100_0.5-2C_b',
]

In [ ]:
import pandas as pd

def fetch_data():
    for name in tables:
        link = r'https://www.batteryarchive.org/data/' + name + r'_timeseries.csv'
        print(f"DONE: {link}")
        df = pd.read_csv(link)
        df.to_pickle(name + '.pkl')

fetch_data()

DONE: https://www.batteryarchive.org/data/SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv
DONE: https://www.batteryarchive.org/data/SNL_18650_LFP_15C_0-100_0.5-1C_b_timeseries.csv
DONE: https://www.batteryarchive.org/data/SNL_18650_LFP_15C_0-100_0.5-2C_a_timeseries.csv
